<a href="https://colab.research.google.com/github/Garima27dec/Emotion-Detection-with-Multiple-Data-Streams/blob/main/Ensemble_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Importing and installing Libraries**

In [3]:
!pip install -U tensorflow-text==2.12.0
!pip3 install opencv-python
!pip install pydub
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 17.6 MB/s eta 0:00:00


In [4]:
import tensorflow_hub as hub
import tensorflow_text as text
import os
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import shutil
import tensorflow as tf
import tensorflow.keras as keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM,Dropout,TimeDistributed, Flatten, Bidirectional, Input
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
import matplotlib.pyplot as plt
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy
import re
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import glob
import statistics
from scipy.ndimage import zoom
from os import path
from pydub import AudioSegment
import librosa
import IPython
from IPython.display import Audio, Image
import speech_recognition

##**Connecting to Storage - to import saved models**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Loading Face, Speech, Text Emotion Recognition Models**

In [6]:
from keras.models import model_from_json
with open('/content/drive/MyDrive/Emotion_Recognition/FER/savedmodels/model_3.json','r') as f:
    json = f.read()
model_fer = model_from_json(json)

model_fer.load_weights('/content/drive/MyDrive/Emotion_Recognition/FER/savedmodels/model_3.h5')
print("Loaded FER model from disk")

Loaded FER model from disk


In [7]:
from keras.models import load_model
model_ser = load_model('/content/drive/MyDrive/Emotion_Recognition/SER/savedmodel/[CNN-LSTM]M.h5')

model_ser.load_weights('/content/drive/MyDrive/Emotion_Recognition/SER/savedmodel/[CNN-LSTM]W.h5')
print("Loaded SER model from disk")

Loaded SER model from disk


In [8]:
from keras.models import load_model
model_ter = tf.keras.models.load_model(('/content/drive/MyDrive/Emotion_Recognition/Text_ER/BERT_M.h5'), custom_objects={'KerasLayer':hub.KerasLayer})
model_ter.load_weights('/content/drive/MyDrive/Emotion_Recognition/Text_ER/BERT_W.h5')
print("Loaded Text_ER model from disk")

Loaded Text_ER model from disk


##**Defining functions for Data Pre-processing**

In [9]:
shape_x = 48
shape_y = 48

In [10]:
def detect_face(frame):

    #Cascade classifier pre-trained model
    cascPath = '/content/drive/MyDrive/Emotion_Recognition/FER/haarcascade_frontalface_default.xml'
    faceCascade = cv2.CascadeClassifier(cascPath)

    #BGR -> Gray conversion
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #Cascade MultiScale classifier
    detected_faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=6,
                                                  minSize=(shape_x, shape_y),
                                                  flags=cv2.CASCADE_SCALE_IMAGE)
    coord = []

    for x, y, w, h in detected_faces :
        if w > 100 :
            sub_img=frame[y:y+h,x:x+w]
            #cv2.rectangle(frame,(x,y),(x+w,y+h),(0, 255,255),1)
            coord.append([x,y,w,h])

    return gray, detected_faces, coord

In [11]:
def extract_face_features(faces, offset_coefficients=(0.075, 0.05)):
    gray = faces[0]
    detected_face = faces[1]

    new_face = []

    for det in detected_face :

        x, y, w, h = det


        #Offset coefficient, np.floor takes the lowest integer (delete border of the image)
        horizontal_offset = np.int(np.floor(offset_coefficients[0] * w))
        vertical_offset = np.int(np.floor(offset_coefficients[1] * h))

        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        extracted_face = gray[y+vertical_offset:y+h, x+horizontal_offset:x-horizontal_offset+w]


        new_extracted_face = zoom(extracted_face, (shape_x / extracted_face.shape[0],shape_y / extracted_face.shape[1]))
        #cast type float
        new_extracted_face = new_extracted_face.astype(np.float32)
        #scale
        new_extracted_face /= float(new_extracted_face.max())
        #print(new_extracted_face)

        new_face.append(new_extracted_face)

    return new_face

In [12]:
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite("/content/drive/MyDrive/Emotion_Recognition/FER/Results/"+"image"+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames


In [13]:
def get_label_fer(argument):
    labels = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad' , 5:'Surprise', 6:'Neutral'}
    return(labels.get(argument, "Invalid emotion"))

In [14]:
def get_label_ser(argument):
    labels = {0:'Neutral', 1:'Happy', 2:'Sad', 3:'Angry', 4:'Fear' , 5:'Disgust', 6:'Surprise'}
    return(labels.get(argument, "Invalid emotion"))

In [15]:
def mel_spectrogram(y, sr=16000, n_fft=512, win_length=256, hop_length=128, window='hamming', n_mels=128, fmax=4000):

    # Compute spectogram
    mel_spect = np.abs(librosa.stft(y, n_fft=n_fft, window=window, win_length=win_length, hop_length=hop_length)) ** 2

    # Compute mel spectrogram
    mel_spect = librosa.feature.melspectrogram(S=mel_spect, sr=sr, n_mels=n_mels, fmax=fmax)

    # Compute log-mel spectrogram
    mel_spect = librosa.power_to_db(mel_spect, ref=np.max)

    return mel_spect

In [16]:
# Time distributed parameters
win_ts = 128
hop_ts = 64

# Split spectrogram into frames
def frame(x, win_step=128, win_size=64):
    nb_frames = 1 + int((x.shape[2] - win_size) / win_step)
    frames = np.zeros((x.shape[0], nb_frames, x.shape[1], win_size)).astype(np.float32)
    for t in range(nb_frames):
        frames[:,t,:,:] = np.copy(x[:,:,(t * win_step):(t * win_step + win_size)]).astype(np.float32)
    return frames

In [17]:
def text_prediction(test_string):
  class_prob = model_ter.predict(test_string, batch_size=1)[0]
  if np.argmax(class_prob)==0:
    return("Angry")
  elif np.argmax(class_prob)==1:
    return("Disgust")
  elif np.argmax(class_prob)==2:
    return("Fear")
  elif np.argmax(class_prob)==3:
    return("Happy")
  elif np.argmax(class_prob)==4:
    return("Neutral")
  elif np.argmax(class_prob)==5:
    return("Sad")
  else:
    return("Surprise")

#**Ensemble Model**

In [89]:
src = "/content/drive/MyDrive/Emotion_Recognition/Model_Testing/MELD.Raw/train_splits/dia0_utt0.mp4"

In [102]:
final_emotion = []
final_probability =[]

src = "/content/drive/MyDrive/Emotion_Recognition/Model_Testing/MELD.Raw/train_splits/dia0_utt0.mp4"

vidcap = cv2.VideoCapture(src)

sec = 0
frameRate = 0.5 #//it will capture image in each 0.5 second
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

emotions = []
emotions_prob = []

source = glob.glob('/content/drive/MyDrive/Emotion_Recognition/FER/Results/*')

for i in source:
  face = cv2.imread(i)
  for face in extract_face_features(detect_face(face)) :
    to_predict = np.reshape(face.flatten(), (1,48,48,1))
    res1 = model_fer.predict(to_predict)
    result_num = np.argmax(res1)
    prob = np.amax(res1)
    emotion_per_frame = get_label_fer(result_num)
    #print (emotion_per_frame)
    emotions.append(emotion_per_frame)
    emotions_prob.append(prob)

#print(emotions)
#print(emotions_prob)
print("Emotion from FER: ", statistics.mode(emotions))
print("Emotion Probability: ", statistics.mean(emotions_prob))

folder_path = '/content/drive/MyDrive/Emotion_Recognition/FER/Results/'

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.jpg'):
            os.remove(os.path.join(root, file))

dst = "/content/drive/MyDrive/Emotion_Recognition/Model_Testing/wav_files/test0.wav"

# convert mp4 to wav
sound = AudioSegment.from_file(src,format="mp4")
sound.export(dst, format="wav")

# Sample rate (16.0 kHz)
sample_rate = 16000

# Max pad lenght (3.0 sec)
max_pad_len = 49100

        # Read audio file
y, sr = librosa.core.load(dst, sr=sample_rate, offset=0.5)

        # Z-normalization
y = zscore(y)

        # Padding or truncated signal
if len(y) < max_pad_len:
  y_padded = np.zeros(max_pad_len)
  y_padded[:len(y)] = y
  y = y_padded
elif len(y) > max_pad_len:
  y = np.asarray(y[:max_pad_len])

# Data Preprocessing
mel_spect = np.asarray(mel_spectrogram(y))
mel_spect = np.reshape(mel_spect,(1,128,384))
input = frame(mel_spect, hop_ts, win_ts)
input = input.reshape(input.shape[0], input.shape[1] , input.shape[2], input.shape[3], 1)

for speech in input:
    res = model_ser.predict(input)
    result_num2 = np.argmax(res)
    #print(result_num)
    print("Emotion from SER:", get_label_ser(result_num2))
    print("Emotion Probability: ", np.amax(res))

r = speech_recognition.Recognizer()

hellow=speech_recognition.AudioFile(dst)
with hellow as source:
    audio = r.record(source)
try:
    s = r.recognize_google(audio)
    print("Text: "+s)
except Exception as e:
    print("Exception: "+str(e))

print("Emotion from Text: ", text_prediction([s]))
print("Emotion Probability: ", np.amax(model_ter.predict([s])))

compile_emotion = [statistics.mode(emotions), get_label_ser(result_num2), text_prediction([s])]
compile_emotion_prob = [statistics.mean(emotions_prob), np.amax(res), np.amax(model_ter.predict([s]))]

if ((compile_emotion[0] != compile_emotion[1]) and (compile_emotion[0] != compile_emotion[2]) and (compile_emotion[1] != compile_emotion[2])):
  idx = compile_emotion_prob.index(max(compile_emotion_prob))
  final_emotion.append(compile_emotion[idx])
  final_probability.append(max(compile_emotion_prob))

else:
  final_emotion.append(statistics.mode(compile_emotion))

if ((compile_emotion[0] == compile_emotion[1]) and (compile_emotion[0] == compile_emotion[2])):
  final_probability.append(statistics.mean(compile_emotion_prob))
elif ((compile_emotion[0] == compile_emotion[1]) and (compile_emotion[0] != compile_emotion[2])) :
  final_probability.append((compile_emotion_prob[0]+compile_emotion_prob[1])/2)
elif ((compile_emotion[0] == compile_emotion[2]) and (compile_emotion[0] != compile_emotion[1])):
  final_probability.append((compile_emotion_prob[0]+compile_emotion_prob[2])/2)
else:
  final_probability.append((compile_emotion_prob[1]+compile_emotion_prob[2])/2)

print("Final Emotion: ", final_emotion)
print("Final Emotion Probability: ", final_probability)

1/1 [==============================] - 0s 50ms/step


<ipython-input-11-62d6636c3fdd>:13: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  horizontal_offset = np.int(np.floor(offset_coefficients[0] * w))
<ipython-input-11-62d6636c3fdd>:14: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.

1/1 [==============================] - 0s 127ms/step
Emotion from FER:  Angry
Emotion Probability:  0.8369248
1/1 [==============================] - 0s 228ms/step
Emotion from SER: Neutral
Emotion Probability:  0.8652872
Text: transition from the kl5 to gr6 system
1/1 [==============================] - 1s 1s/step
Emotion from Text:  Neutral
1/1 [==============================] - 1s 976ms/step
Emotion Probability:  0.43641794
1/1 [==============================] - 1s 840ms/step
Final Emotion:  ['Neutral']
Final Emotion Probability:  [0.6508525609970093]
